<a href="https://colab.research.google.com/github/Saifullah785/Agentic_AI_LangGraph_Learning_Journey/blob/main/Parallel_Workflows_in_LangGraph/UPSC_essay_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ! pip install langchain_openai
# ! pip install langgraph

In [5]:
# Import necessary libraries
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [6]:
from google.colab import userdata
OPENAI_API_KEY= userdata.get('OPENAI_API_KEY')


In [7]:
# Load environment variables

#load_dotenv()

In [8]:
# Initialize the ChatOpenAI model
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [9]:
# Define the Pydantic schema for evaluation
class EvaluationSchema(BaseModel):

    feedback: str = Field(description='Detailed feedbackfor the essay')
    score: int = Field(description='Score out of 10', ge=0, le=10)

In [10]:
# Structure the model output using the defined schema
structured_model = model.with_structured_output(EvaluationSchema)

/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1913: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [11]:
# Define the first essay for evaluation
essay = """India in the Age of AI
as the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.

india's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. with over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for All) with a focus on inclusive growth, aiming to leverage AI in healthcare, agriculture, education, and smart mobility.

one of the most promising applications of AI in India lies in agriculture, where predictive analytics can guide farmers on optimal sowing times, weather forecasts, and pest control. In healthcare, AI-powered diagnostics can help address India’s doctor-patient ratio crisis, particularly in rural areas. Educational platforms are increasingly using AI to personalize learning paths, while smart governance tools are helping improve public service delivery and fraud detection.

However, the path to AI-led growth is riddled with challenges. Chief among them is the digital divide. While metropolitan cities may embrace AI-driven solutions, rural India continues to struggle with basic internet access and digital literacy. The risk of job displacement due to automation also looms large, especially for low-skilled workers. Without effective skilling and re-skilling programs, AI could exacerbate existing socio-economic inequalities.

Another pressing concern is data privacy and ethics. as AI systems rely heavily on vast datasets, ensuring that personal data is used transparently and responsibly becomes vital. India is still shaping its data protection laws, and in the absence of a strong regulatory framework, AI systems may risk misuse or bias.

to harness AI responsibly, India must adopt a multi-stakeholder approach involving the government, academia, industry, and civil society. Policies should promote open datasets, encourage responsible innovation, and ensure ethical AI practices. There is also a need for international collaboration, particularly with countries leading in AI research, to gain strategic advantage and ensure interoperability in global systems.

India’s demographic dividend, when paired with responsible AI adoption, can unlock massive economic growth, improve governance, and uplift marginalized communities. But this vision will only materialize if AI is seen not merely as a tool for automation, but as an enabler of human-centered development.

In conclusion, India in the age of AI is a story in the making — one of opportunity, responsibility, and transformation. The decisions we make today will not just determine India’s AI trajectory, but also its future as an inclusive, equitable, and innovation-driven society."""

In [12]:
# Evaluate the language quality of the first essay
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
structured_model.invoke(prompt).feedback

"The essay on 'India in the Age of AI' provides a comprehensive overview of the opportunities and challenges presented by artificial intelligence in India. The language used is clear and coherent, effectively conveying the key points. The essay is well-structured with a logical flow of ideas. However, there are some minor grammatical errors and improvements can be made in transitioning between paragraphs for smoother readability. Overall, the content is informative and engaging, showcasing a good understanding of the topic."

In [13]:
# Define the state for the UPSC evaluation graph
class UPSCState(TypedDict):

    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [14]:
# Node function to evaluate language quality
def evaluate_language(state: UPSCState):

    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'language_feedback': output.feedback, 'individual_scores': [output.score]}

In [15]:
# Node function to evaluate depth of analysis
def evaluate_analysis(state: UPSCState):

    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'analysis_feedback': output.feedback, 'individual_scores': [output.score]}

In [16]:
# Node function to evaluate clarity of thought
def evaluate_thought(state: UPSCState):

    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'clarity_feedback': output.feedback, 'individual_scores': [output.score]}

In [17]:
# Node function for final evaluation and score calculation
def final_evaluation(state: UPSCState):

    # summary feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content

    # avg calculate
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

In [18]:
# Define and compile the StateGraph for the evaluation workflow
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)

# edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

workflow = graph.compile()

In [19]:
# Display the compiled workflow
workflow

In [20]:
# Define the second essay for evaluation
essay2 = """Pakistan, AI Technology, and Military Comparison with the United States

Artificial Intelligence (AI) has become a defining force of the 21st century, influencing economies, societies, and defense strategies. Pakistan, though a developing country, is gradually making progress in AI, while the United States remains the world leader in research, adoption, and military integration.

Pakistan has contributed to AI by establishing the National Centre of Artificial Intelligence (NCAI), encouraging research in robotics, computer vision, and natural language processing. Universities such as NUST, FAST, and LUMS are producing talent that applies AI in healthcare, agriculture, and fintech. The government’s National AI Policy Framework also aims to create an AI-ready workforce. Although Pakistan’s contribution is modest compared to global leaders, its focus on human capital and sector-specific applications reflects steady progress.

In contrast, the United States dominates AI research and commercialization through institutions like MIT and Stanford, alongside global tech giants such as Google and Microsoft. AI tools such as TensorFlow and PyTorch, widely used in Pakistan, are American innovations. Thus, while the U.S. integrates AI into everyday industries at scale, Pakistan primarily adopts U.S. and Chinese frameworks to build its foundation.

The most significant contrast appears in military adoption. The U.S. employs AI in projects like Project Maven and the Skyborg program, developing autonomous drones, missile defense systems, and predictive battlefield intelligence. AI enhances real-time decision-making and ensures air superiority. Pakistan, meanwhile, incorporates AI into radar systems, surveillance, and UAVs, largely through collaborations with China, such as acquiring AI-enabled drones. Its progress is promising but limited by smaller budgets and reliance on foreign technology.

In conclusion, Pakistan is laying the groundwork for AI integration, while the United States represents technological leadership. Bridging this gap requires Pakistan to expand indigenous research, defense investment, and international collaboration to secure its economic and strategic future."""

In [21]:
# Invoke the workflow with the second essay
intial_state = {
    'essay': essay2
}

workflow.invoke(intial_state)

{'essay': 'Pakistan, AI Technology, and Military Comparison with the United States\n\nArtificial Intelligence (AI) has become a defining force of the 21st century, influencing economies, societies, and defense strategies. Pakistan, though a developing country, is gradually making progress in AI, while the United States remains the world leader in research, adoption, and military integration.\n\nPakistan has contributed to AI by establishing the National Centre of Artificial Intelligence (NCAI), encouraging research in robotics, computer vision, and natural language processing. Universities such as NUST, FAST, and LUMS are producing talent that applies AI in healthcare, agriculture, and fintech. The government’s National AI Policy Framework also aims to create an AI-ready workforce. Although Pakistan’s contribution is modest compared to global leaders, its focus on human capital and sector-specific applications reflects steady progress.\n\nIn contrast, the United States dominates AI res